In [5]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [48]:
class SART(object):
    
    def __init__(self, r0_seeds, h_pairs, fd_into_ht_pairs, columns):
        self.r0_seeds = r0_seeds
        self.h_pairs = h_pairs
        self.fd_into_ht_pairs = fd_into_ht_pairs
        self.columns = columns
        self.h_seed_key = defaultdict(lambda: [], self.r0_seeds)
        self.bef_aft_ht_pairs = zip(self.h_pairs, self.fd_into_ht_pairs)
        self.sd_h_key = None
        self.bracket_df = None
        
    def _define_ht_sd_keys(self, pri_heat_pair1, pri_heat_pair2, win_heat, lose_heat):
        self.h_seed_key['Heat {}'.format(win_heat)].append('W{}'.format(pri_heat_pair1))
        self.h_seed_key['Heat {}'.format(win_heat)].append('W{}'.format(pri_heat_pair2))
        
        for i in range(1,4):
            self.h_seed_key['Heat {}'.format(win_heat)].append('{}/{}-Q{}'.format(pri_heat_pair1, pri_heat_pair2, i))
            
        for i in range(4,9):
            self.h_seed_key['Heat {}'.format(lose_heat)].append('{}/{}-Q{}'.format(pri_heat_pair1, pri_heat_pair2, i))
    
    def _assign_ht_sd_keys(self):
        for pri_heats, wn_ls_heats in self.bef_aft_ht_pairs:
            pri_heat1 = pri_heats[0]
            pri_heat2 = pri_heats[1]
            win_ht = wn_ls_heats[0]
            lose_ht = wn_ls_heats[1]
            self._define_ht_sd_keys(pri_heat1, pri_heat2, win_ht, lose_ht)
    
    def _assign_flip_sd_ht_keys(self):
        self.sd_h_key = {val:key for key, value in self.h_seed_key.iteritems() for i, val in enumerate(value)}
    
    def prepare_sd_h_keys(self):
        self._assign_ht_sd_keys()
        self._assign_flip_sd_ht_keys()
    
    def create_bracket(self):
        RH_index = [i for i in self.sd_h_key.keys()]
        self.bracket_df = pd.DataFrame(index=RH_index, columns=self.columns)
        self.bracket_df.index.name = 'Seed'
        
        for val in self.bracket_df.index:
            self.bracket_df.loc[val,['Nxt_Heat']] = self.sd_h_key[val]
            self.bracket_df.loc[val,['Round']] = int(str(self.bracket_df.loc[val,['Nxt_Heat']]).split()[2][0])-1
            
        self.bracket_df.sort_index(inplace=True)

In [51]:
r0_seeds = {'Heat 101': [1, 32, 33, 64, 65], 'Heat 102': [16, 17, 48, 49, 80],
    'Heat 103': [8, 25, 40, 57, 72], 'Heat 104': [9, 24, 41, 56, 73],
    'Heat 105': [4, 29, 36, 61, 68], 'Heat 106': [13, 20, 45, 52, 77],
    'Heat 107': [5, 28, 37, 60, 69], 'Heat 108': [12, 21, 44, 53, 76],
    'Heat 109': [2, 31, 34, 63, 66], 'Heat 110': [15, 18, 47, 50, 79],
    'Heat 111': [7, 26, 39, 58, 71], 'Heat 112': [10, 23, 42, 55, 74],
    'Heat 113': [3, 30, 35, 62, 67], 'Heat 114': [14, 19, 46, 51, 78],
    'Heat 115': [6, 27, 38, 59, 70], 'Heat 116': [11, 22, 43, 54, 75]}

    #heat pairs round 1 - 4:
h_pairs = [(101, 102), (103, 104), (105, 106), (107, 108), (109, 110),
    (111, 112), (113, 114), (115, 116), (201, 203), (205, 207), (209, 211),
    (213, 215), (202, 204), (206, 208), (210, 212), (214, 216), (301, 302),
    (303, 304), (305, 306), (307, 308), (309, 310), (311, 312), (313, 314),
    (315, 316), (401, 402), (403, 404), (405, 406), (407, 408), (409, 410),
    (411, 412), (413, 414), (415, 416)]

    #heat pairs round 2 - 5 that prior round feed into e.g. 101, 102 feed into 201, 202:
fd_into_ht_pairs = [(201, 202), (203, 204), (205, 206), (207, 208),
    (209, 210), (211, 212), (213, 214), (215, 216), (316, 312), (315, 311),
    (314, 310), (313, 309), (308, 304), (307, 303), (306, 302), (305, 301),
    (403, 401), (404, 402), (407, 405), (408, 406), (411, 409), (412, 410),
    (415, 413), (416, 414), (502, 501), (504, 503), (506, 505), (508, 507),
    (510, 509), (512, 511), (514, 513), (516, 515)]

    #Bracket columns:
columns = ['Round', 'Nxt_Heat', 'First_name', 'Surname', 'Time', 'Nxt_Heat_Time']

In [52]:
sart_2017 = SART(r0_seeds, h_pairs, fd_into_ht_pairs, columns)

In [53]:
sart_2017.prepare_sd_h_keys()

In [54]:
sart_2017.create_bracket()

In [56]:
sart_2017.bracket_df.head()

,Round,Nxt_Heat,First_name,Surname,Time,Nxt_Heat_Time
Seed,,,,,,
1,0,Heat 101,NaN,NaN,NaN,NaN
2,0,Heat 109,NaN,NaN,NaN,NaN
3,0,Heat 113,NaN,NaN,NaN,NaN
4,0,Heat 105,NaN,NaN,NaN,NaN
5,0,Heat 107,NaN,NaN,NaN,NaN
